# Advanced analysis of estimates

👋 Hello, this sample will showcase how to perform resource estimation
experiments created on top of Azure Quantum Resource Estimator.  The sample will
re-use the quantum multiplication algorithm from the _Estimates with Q#_
notebook. We leverage the `azure_quantum` API to configure and submit resource
estimation jobs.

## Setup

Let's import some packages and connect to the Azure Quantum workspace.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator, QubitParams
from azure.quantum.target.microsoft.target import MicrosoftEstimatorQubitParams
import numpy as np                         # To store experimental data from job results
from matplotlib import pyplot as plt       # To plot experimental results
from matplotlib.colors import hsv_to_rgb   # To automatically find colors for plots

In [ ]:
workspace = Workspace(
    resource_id = "",
    location = "",
)

We are also importing the `qsharp` Python package to write Q# code in Jupyter
cells, and import the `Microsoft.Quantum.Numerics` package that contains the
multiplication algorithm.

In [ ]:
import qsharp
qsharp.packages.add("Microsoft.Quantum.Numerics")

## Implementing the algorithm

As running example algorithm we are creating a multiplier using the [MultiplyI](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arithmetic.multiplyi) operation.  We can configure the size of the multiplier with a bitwidth parameter. The operation will have two input registers with that bitwidth, and one output register with the size of twice the bitwidth.

In [ ]:
EstimateMultiplication: any = None # Make Python recognize the Q# function (optional)

In [ ]:
%%qsharp

open Microsoft.Quantum.Arithmetic;

operation EstimateMultiplication(bitwidth : Int) : Unit {
    use factor1 = Qubit[bitwidth];
    use factor2 = Qubit[bitwidth];
    use product = Qubit[2 * bitwidth];

    MultiplyI(LittleEndian(factor1), LittleEndian(factor2), LittleEndian(product));
}

## Setting up and running the experiments

Next, we are setting up some experiments. Here, we are using two of the six
pre-defined qubit parameter models, and one customized model based on the model
`qubit_gate_ns_e3` (accessed via the constant `QubitParams.GATE_NS_E3`), in
which we change the error rates to $10^{-3.5} \approx 0.00032$. In your own
experiments, you can change the number of items, and also the parameters. You
may use other pre-defined models or define custom models. You can find more
information about the input parameters in the _Getting Started with Azure
Quantum Resource Estimation_ notebook.  Note that in `target_params` we have
tuples of names and qubit parameters.  We are using the names for the plots when
analyzing the results.

Further, we are choosing a list of input parameters to our algorithm, in this
case bitwidths that are powers-of-2 ranging from 8 to 64.

In [ ]:
estimator = MicrosoftEstimator(workspace=workspace)

target_params = [
    ("Gate-based ns, 10⁻³", MicrosoftEstimatorQubitParams(name=QubitParams.GATE_NS_E3)),
    ("Gate-based ns, 10⁻³ᐧ⁵", MicrosoftEstimatorQubitParams(name=QubitParams.GATE_NS_E3, one_qubit_measurement_error_rate=0.00032, one_qubit_gate_error_rate=0.00032, two_qubit_gate_error_rate=0.00032, t_gate_error_rate=0.00032)),
    ("Gate-based ns, 10⁻⁴", MicrosoftEstimatorQubitParams(name=QubitParams.GATE_NS_E4))
]
bitwidths = [8, 16, 32, 64]

# This is to access the names of the target parameters
names = [name for (name, _) in target_params]

We are now creating a batching job, in which we create all the combination of
target_params and bitwidths.  The bitwidth is assigned by accessing it through
the `arguments` field of the item.  We then submit the job with these items for
the multiplication algorithm and wait for the results.

In [ ]:
params = estimator.make_params(num_items=len(bitwidths) * len(target_params))

for i, (_, target_param) in enumerate(target_params):
    for j, bitwidth in enumerate(bitwidths):
        params.items[i * len(bitwidths) + j].qubit_params = target_param
        params.items[i * len(bitwidths) + j].arguments["bitwidth"] = bitwidth

job = estimator.submit(EstimateMultiplication, input_params=params)
results = job.get_results()

## Plotting the experimental results

Now that we have computed the results, we extract some data from it.  We extract the
number of physical qubits, the total runtime in nanoseconds, and the QEC code
distance for the logical qubits.  In addition to the total number of physical
qubits, we are also extracting their breakdown into number of physical qubits
for executing the algorithm and the number of physical qubits required for the T
factories that produce the required T states.

In [ ]:
qubits = np.zeros((len(names), len(bitwidths), 3))
runtime = np.zeros((len(names), len(bitwidths)))
distances = np.zeros((len(names), len(bitwidths)))

for bitwidth_index, bitwidth in enumerate(bitwidths):
    for name_index, name in enumerate(names):
        # Note that the results are ordered by target parameters first, then by bitwidth
        data = results.data(name_index * len(bitwidths) + bitwidth_index)

        qubits[(name_index, bitwidth_index, 0)] = data['physicalCounts']['physicalQubits']
        qubits[(name_index, bitwidth_index, 1)] = data['physicalCounts']['breakdown']['physicalQubitsForAlgorithm']
        qubits[(name_index, bitwidth_index, 2)] = data['physicalCounts']['breakdown']['physicalQubitsForTfactories']

        runtime[(name_index, bitwidth_index)] = data['physicalCounts']['runtime']

        distances[(name_index, bitwidth_index)] = data['logicalQubit']['codeDistance']

Finally, we are using [Matplotlib](https://matplotlib.org/) to plot the number
of physical qubits and the runtime as bar plots, and the QEC code distances as a
scatter plot.  For the physical qubits, we are showing the partition into qubits
required for the algorithm and qubits required for the T factories.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(22, 6))

num_experiments = len(names)                         # Extract number of experiments form names (can be made smaller)
xs = np.arange(0, len(bitwidths))                    # Map bitwidths to numeric indexes for plotting
full_width = .8                                      # Total width of all bars (should be smaller than 1)
width = full_width / num_experiments                 # Fractional width of a single bar
xs_left = xs - (((num_experiments - 1) * width) / 2) # Starting x-coordinate for bars

# Split axes into qubit and runtime plots
ax_qubits, ax_runtime, ax_code_distance = axs

# Plot physical qubits
for i in range(num_experiments):
    ax_qubits.bar(xs_left + i * width, qubits[i,:,1], width, label=f"{names[i]} (Alg.)", color=hsv_to_rgb((i / num_experiments, 1.0, .8)))
    ax_qubits.bar(xs_left + i * width, qubits[i,:,2], width, bottom=qubits[i,:,1], label=f"{names[i]} (T fac.)", color=hsv_to_rgb((i / num_experiments, 0.3, .8)))
ax_qubits.set_title("#Physical qubits")
ax_qubits.set_xlabel("Bitwidth")
ax_qubits.set_xticks(xs)
ax_qubits.set_xticklabels(bitwidths)
ax_qubits.legend()

# Plot runtime
for i in range(num_experiments):
    ax_runtime.bar(xs_left + i * width, np.array(runtime[i,:]) / 1e6, width, label=names[i], color=hsv_to_rgb((i / num_experiments, 1.0, .8)))
ax_runtime.set_title("Runtime (ms)")
ax_runtime.set_xlabel("Bitwidth")
ax_runtime.set_xticks(xs)
ax_runtime.set_xticklabels(bitwidths)
ax_runtime.legend()

# Plot code distances
for i in range(num_experiments):
    ax_code_distance.scatter(xs, distances[i,:], label=names[i], marker='*', color=hsv_to_rgb((i / num_experiments, 1.0, 0.8)))
ax_code_distance.set_title("QEC code distance")
ax_code_distance.set_xlabel("Bitwidth")
ax_code_distance.set_xticks(xs)
ax_code_distance.set_xticklabels(bitwidths)
ax_code_distance.legend()

fig.suptitle("Resource estimates for multiplication")
plt.show()

## Accessing the results table

Next, we are showing all estimation results for the first bitwidth in a
side-by-side table.  Notice that the items are ordered by target parameters
first, then by bitwidths.  Therefore, all items with bitwidth 8 are at indices
0, 4, and 8, where the step size 4 corresponds to the number of different
bitwidths.

You can also access individual results by providing a number as index, e.g.,
`results[1]` to show the results table of the configuration with the first set
of target parameters and bitwidth 16.

In [ ]:
bitwidth_index = 0
results[bitwidth_index::len(bitwidths)]

Further, we can plot all items in the result object using the `plot()` function.
That function takes as optional parameter an array of labels for the plot's
legend.  We are deriving the labels from the names and bitwidths, similar to how
the items were created.

In [ ]:
results.plot(labels=[f"{name} ({bitwidth} bit)" for name in names for bitwidth in bitwidths])

## Next steps

We hope you got some ideas and inspirations for your own resource estimation
experiments.  Feel free to use this notebook as a starting point for your own
algorithm investigations.  To get more familiar with resource estimation
experiments, here are some suggestions to try out in this notebook:

* Add a fourth plot to show some statistics about a single T factory, e.g., its
  number of qubits.
* Add a new plot series to show logical resource estimates.
* Change the algorithm to create an $n$-ary multiplier (with a variable number
  of input arguments) for either a fixed or customizable bitwidth.
* Create a side-by-side comparison table for a fixed qubit parameter comparing all bitwidths.